<a href="https://colab.research.google.com/github/Thiago-Emidio-Silva/artigo_Levantamento-de-caracter-sticas-de-coagula-o-para-tratamento-de-gua-pot-vel/blob/main/Treinamento_CNN_Artigo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Levantamento de características de coagulação para tratamento de água potável**

Thiago Emidio da Silva – 118772
Departamento de Engenharia Elétrica – Especialização em Inteligência Artificial e Computacional
Universidade Federal de Viçosa. Viçosa – MG -- Thiago.emidio@ufv.br


VAMOS CARREGAR AS BIBLIOTECAS NECESSÁRIAS PARA NOSSA APLICAÇÃO DE TREINAMENTO DO MODELO

In [1]:
#!pip install tensorflow --upgrade


In [2]:
#Permição para acessar o google drive se necessário
from google.colab import drive
drive.mount('/content/drive')

#Carregar as bibliotecas necessarias
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from google.colab.patches import cv2_imshow
import os
from skimage.io import imread, imshow
from scipy import stats
from google.colab import output
from IPython.display import display, HTML, clear_output
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, UpSampling2D, Conv2DTranspose
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.losses import SparseCategoricalCrossentropy

#from tensorflow_examples.models.pix2pix import pix2pix



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


VAMOS DEFINIR AS PROPRIEDADES DAS IMAGENS


In [3]:
#Vamos ler a imagem

# Caminho da pasta de imagens no Google Drive ou local do Colab
pasta_imagens_train = '/content/drive/MyDrive/img_floc_256/train/'
pasta_imagens_test = '/content/drive/MyDrive/img_floc_256/test/'

imagemSize = (256, 256)
bathSize = 32

In [20]:
# Função de normalização e codificação
def normalizar(input_imagem, input_mask, num_classes=3):
    # Normalizar imagem de entrada para o intervalo [0, 1]
    input_imagem = tf.cast(input_imagem, tf.float32) / 255.0

    # Ajustar máscara para ter valores de classe entre 0 e num_classes-1
    input_mask = cv2.cvtColor(input_mask, cv2.COLOR_BGR2GRAY)  # Converte para escala de cinza
    input_mask = input_mask // (255 // (num_classes - 1))  # Ajusta valores para 0, 1, 2 (exemplo com 3 classes)

    #input_mask = input_mask.astype(np.int32)  # Ajuste para valores entre 0 e 2

    # Converte máscara para codificação one-hot
    #input_mask = tf.one_hot(input_mask, depth=num_classes)

    return input_imagem, input_mask

# Função para carregar a imagem
def load_imagem(pasta_imagens_train, nome_arquivo, imagemSize):
    caminho_imagem = os.path.join(pasta_imagens_train, f"{nome_arquivo}.jpeg")
    input_imagem = cv2.imread(caminho_imagem, cv2.IMREAD_COLOR)
    input_imagem = cv2.resize(input_imagem, imagemSize)

    caminho_mask = os.path.join(pasta_imagens_train, f"{nome_arquivo}.png")
    input_mask = cv2.imread(caminho_mask, cv2.IMREAD_COLOR)
    input_mask = cv2.resize(input_mask, imagemSize)

    return input_imagem, input_mask

# Função para carregar o dataset
def load_dataset(pasta_imagens_train, imagemSize, num_classes=3):
    dataset = []

    arquivos = os.listdir(pasta_imagens_train)
    nomes_imagens = [f.split(".")[0] for f in arquivos if f.endswith(".jpeg")]

    for nome_arquivo in nomes_imagens:
        input_imagem, input_mask = load_imagem(pasta_imagens_train, nome_arquivo, imagemSize)
        input_imagem, input_mask = normalizar(input_imagem, input_mask, num_classes=num_classes)
        dataset.append((input_imagem, input_mask))

    return dataset

# Listar arquivos de imagem
arquivos = [f for f in os.listdir(pasta_imagens_train) if f.lower().endswith(('.jpeg'))]
indice_atual = 0  # Iniciar o índice da primeira imagem

# Carregar dataset
dataset = load_dataset(pasta_imagens_train, imagemSize)

# Dividir dados em treino e validação
input_imagens, input_masks = zip(*dataset)
x_train, x_val, y_train, y_val = train_test_split(input_imagens, input_masks, test_size=0.2, random_state=42)

# Converter para arrays numpy
x_train = np.array(x_train)
x_val = np.array(x_val)
y_train = np.array(y_train)
y_val = np.array(y_val)

print("Formato x_train:", x_train.shape)  # Esperado: (N, 256, 256, 3)
print("Formato y_train:", y_train.shape)  # Esperado: (N, 256, 256, 3) para cada classe


Formato x_train: (240, 256, 256, 3)
Formato y_train: (240, 256, 256)


VAMOS DIVIDIR NOSSO DATASET DE TREINO EM TREINAMENTO E VALIDAÇÃO

VAMOS CARREGAR O MODELO RESNET50 PRÉ TREINADO

In [21]:
#Vamos carregar o modelo pré treinado com imagenet
pre_model = ResNet50(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

#Definimos que as camadas do modelo pré treinado não sejam alteradas durante o treino
for layer in pre_model.layers:
    layer.trainable = False

VAMOS DEFINIR O MODELO SEQUENCIAL ONDE TEREMOS O MODELO PRÉTREINADO SEQUIDO DAS NOVAS CAMADAS

In [22]:
# Definição do modelo do tipo sequencial
model = Sequential()

# Adicionando o modelo pré-treinado na primeira camada (como extrator de features)
model.add(pre_model)

# Adicionando camadas convolucionais para segmentação
model.add(Conv2D(256, (3, 3), activation='relu', padding = 'same'))  # Manter dimensões
model.add(Dropout(0.2))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Dropout(0.2))

# Camada final com 3 canais para a segmentação de 3 classes, ativação softmax
model.add(Conv2D(3, (1, 1), activation='softmax', padding='same'))

# Em vez de UpSampling2D, você pode adicionar camadas Conv2DTranspose como abaixo
model.add(Conv2DTranspose(128, (3, 3), strides=(2, 2), padding='same', activation='relu'))
model.add(Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same', activation='relu'))
model.add(Conv2DTranspose(32, (3, 3), strides=(2, 2), padding='same', activation='relu'))
model.add(Conv2DTranspose(16, (3, 3), strides=(2, 2), padding='same', activation='relu'))
model.add(Conv2DTranspose(8, (3, 3), strides=(2, 2), padding='same', activation='relu'))
model.add(Conv2DTranspose(3, (3, 3), strides=(2, 2), padding='same', activation='relu'))

# Resumo do modelo
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)                │ (None, 8, 8, 2048)          │      23,587,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_16 (Conv2D)                   │ (None, 8, 8, 256)           │       4,718,848 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_12 (Dropout)                 │ (None, 8, 8, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_17 (Conv2D)                   │ (None, 6, 6, 128)           │         295,040 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_13 (Dropout)                 │ (None, 6, 6, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_18 (Conv2D)                   │ (None, 4, 4, 64)            │          73,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_14 (Dropout)                 │ (None, 4, 4, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_19 (Conv2D)                   │ (None, 4, 4, 3)             │             195 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose_24                  │ (None, 8, 8, 128)           │           3,584 │
│ (Conv2DTranspose)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose_25                  │ (None, 16, 16, 64)          │          73,792 │
│ (Conv2DTranspose)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose_26                  │ (None, 32, 32, 32)          │          18,464 │
│ (Conv2DTranspose)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose_27                  │ (None, 64, 64, 16)          │           4,624 │
│ (Conv2DTranspose)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose_28                  │ (None, 128, 128, 8)         │           1,160 │
│ (Conv2DTranspose)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose_29                  │ (None, 256, 256, 3)         │             219 │
│ (Conv2DTranspose)                    │                             │                 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 28,777,430 (109.78 MB)

 Trainable params: 5,189,718 (19.80 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

VAMOS EFETUAR O TREINAMENTO DO NOSSO MODELO

In [ ]:
model.compile(optimizer = Adam(learning_rate = 0.00001),
              #loss=SparseCategoricalCrossentropy(from_logits=False),

              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])

history = model.fit(x_train, y_train, batch_size = bathSize, epochs = 10, validation_data = (x_val, y_val))

Epoch 1/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 163s 12s/step - accuracy: 0.3110 - loss: 3.0849 - val_accuracy: 0.5261 - val_loss: 1.9011
Epoch 2/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 145s 13s/step - accuracy: 0.5532 - loss: 1.8048 - val_accuracy: 0.7158 - val_loss: 1.0897
Epoch 3/10
1/8 ━━━━━━━━━━━━━━━━━━━━ 1:05 9s/step - accuracy: 0.6902 - loss: 1.2069

VERIFICANDO O TREINAMENTO DO MODELO

In [ ]:
# Plotar a perda e precisão durante o treinamento
plt.figure(figsize=(12, 4))

# Gráfico da perda
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Loss de Treino')
plt.plot(history.history['val_loss'], label='Loss de Validação')
plt.xlabel('Épocas')
plt.ylabel('Loss')
plt.legend()

# Gráfico da acurácia
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Acurácia de Treino')
plt.plot(history.history['val_accuracy'], label='Acurácia de Validação')
plt.xlabel('Épocas')
plt.ylabel('Acurácia')
plt.legend()

plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

# Definir um mapa de cores para cada classe
COLOR_MAP = np.array([
    [255, 0, 0],   # Classe 0 - Vermelho
    [0, 255, 0],   # Classe 1 - Verde
    [0, 0, 255]    # Classe 2 - Azul
])

def create_mask(pred_mask):
    # Escolha a classe com maior probabilidade para cada pixel
    pred_mask = tf.argmax(pred_mask, axis=-1)
    pred_mask = tf.squeeze(pred_mask, axis=0)  # Remover a dimensão do lote
    pred_mask = pred_mask.numpy().astype(int)  # Converter para int

    # Aplicar o mapa de cores a cada pixel da máscara
    pred_mask_color = COLOR_MAP[pred_mask]
    return pred_mask_color

def show_predictions(input_imagens, input_mask):
    pred_mask = model.predict(input_imagens[tf.newaxis, ...])
    pred_mask_color = create_mask(pred_mask)

    # Exibir a imagem original, a máscara original e a máscara prevista
    fig, ax = plt.subplots(1, 3, figsize=(15, 5))
    ax[0].imshow(input_imagens)
    ax[0].set_title("Imagem Original")
    ax[1].imshow(input_mask)
    ax[1].set_title("Máscara Original")
    ax[2].imshow(pred_mask_color)
    ax[2].set_title("Máscara Prevista")
    for a in ax:
        a.axis("off")
    plt.show()

# Selecionando a primeira imagem e máscara do seu dataset
test_image, test_mask = input_imagens[0], input_masks[0]

# Chamando a função com a imagem e a máscara de teste
show_predictions(test_image, test_mask)
